<div style="background-image: url('https://cdn-images-1.medium.com/max/1200/1*OOWSoWHeQ5kyJ4N0P2ptNA.png');">
<p style="text-align: center;">
  <a href="https://itb.ac.id/">
    <img src="https://ditsti.itb.ac.id/wp-content/uploads/2020/11/logo_itb_1024_bw.png" alt="Logo" width=72 height=72>
  </a>

  <h3 style="text-align: center;">IF3270 Machine Learning</h3>

  <p style="text-align: center;">
    Mengimplementasikan Algoritma FFNN
    <br style="text-align: center;">
    <br style="text-align: center;">
    <!-- <a href="https://reponame/issues/new?template=bug.md">Developer</a>
    ·
    <a href="https://reponame/issues/new?template=feature.md&labels=feature">Table of Contents</a> -->
  </p>
</p>
</div>

In [ ]:
!pip install -r requirements.txt

In [ ]:
# import library
import json
import math
import numpy as np

In [ ]:
# read model file from filesystem
f = open("model.json", "r")

# loads model and parse to json object
model = json.loads(f.read())

network_depth = model["Network_Depth"]
learning_rate = model["Learning_Rate"]
num_of_input = model["Num_Of_Input"]
layer_group = model["Layers"]
weight_group = model["Weights"]
input_group = model["Input"]

In [ ]:
class Layer:
    def __init__(self, width, activation=None):
        '''
        [Attributes]
            _width      : int       { banyak node pada layer }
            _activation : func      { fungsi aktivasi layer, None jika layer input }
            nodes       : list[float] / list[int] { nilai setiap node pada layer }
        '''
        self._width = width
       
        # Set the activation function self._activation = None
        if activation is not None:
            if activation == "linear":
                self._activation = Layer.linear
            if activation == "sigmoid":
                self._activation = Layer.sigmoid
            if activation == "relu":
                self._activation = Layer.relu
            if activation == "softmax":
                self._activation = Layer.softmax
        
        self.nodes = [0] * self._width
    
    def set_nodes(self, val=0):
        '''
        [DESC]
            Menginisialisasi nilai setiap node pada layer
        [PARAMS]
            val     : int/float         { nilai yang akan diinisialisasi untuk setiap node pada layer }
            val     : list[int]|list[float]
        '''
        if isinstance(val, (int, float)):
            self.nodes = [val] * self._width
        if isinstance(val, list):
            self.nodes = val
    
    '''Activation Functions'''
    def activate(self):
        '''
        [DESC]
            Abstraksi yang dipanggil di NeuralNetwork untuk melakukan
            fungsi aktivasi pada setiap node di layer ini
        '''
        self._activation()

    def sigmoid(self):
        '''
        [DESC]
            Mengubah attr nodes dari perkalian vektor weight dengan node
            menjadi list dengan rentang nilai 0 - 1
        [PARAMS]
            
        '''
        for i, z in enumerate(self.nodes):
            self.nodes[i] = 1 / (1 + math.exp(-z))
    
    def linear(self):
        '''
        [DESC]
            Mengubah attr nodes dari perkalian vektor weight dengan node
        [PARAMS]
            X   : list[float]     { input node }
            W   : list[float]     { weight setiap node }
        '''
        pass

    def relu(self):
        '''
        [DESC]
            Menghitung hasil fungsi aktivasi relu.
            Jika nilai dibawah 0 maka kembalian adalah 0, Jika tidak maka mengembalikan nilai dari fungsi linear.
        [PARAMS]
            X   : nilai dari masukan dalam bentuk array of array (Matrix)
            W   : weight dalam bentuk array of array (Matrix)
        '''
        for i, z in enumerate(self.nodes):
            self.nodes[i] = max(0, z)
    
    def softmax(self):
        '''
        [DESC]
            Mengubah attr nodes dengan nilai perbandingan seluruh node
        '''
        temp_z_exps = [np.exp(zi) for zi in self.nodes] # calculate exp(z) for all node in layer
        self.sum_exp_z = np.sum(temp_z_exps)
        for i, z in enumerate(temp_z_exps):
            self.nodes[i] = z / self.sum_exp_z

In [ ]:
class NeuralNetwork:
    def __init__(self, data):
        '''
        [ATTRIBUTES]
            n_
        Nilai data diisi dari model'''
        n_layer = data["Network_Depth"]
        n_input = data["Num_Of_Input"]
        
        self._layers = []
        for i, layer in enumerate(data["Layers"]):
            assert i == layer["depth"]  # check that the json file did not miss a layer
            n_node = layer["width"]
            activation = layer["activation"]
            temp_layer = Layer(width=n_node, activation=activation)
            self._layers.append(temp_layer)
        
        self._weights = []
        for i, weight in enumerate(data["Weights"]):
            assert i == weight["depth_origin"]
            self._weights[i] = weight["values"]

    
    def feed_forward(self, input_group):
        '''
        [DESC]
            Melakukan proses feed forward
        [PARAMS]
            input       : input dari file hmm atau dalam bentuk list atau Layer(?)
        [RETURN]
            output      : Layer     { layer output } ... (1)
            output      : list[int|float]     { list } ... (2)
            # belum diputuskan yg mana 1/2
        '''
        hasil = [0] * num_of_input
        for i, masukan in enumerate(input_group):
            masukan_temp = masukan
            for j in range(network_depth - 1):
                layer = self._layers[j]
                w = np.array(weight_group[j].values) # array of weight
                x = np.array(masukan_temp) # array of input from lower layer
                sigma = np.matmul(w,x)            # matrix multiplication
                layer.nodes = sigma.toList()      # assignment from result multiplication into array nodes
                '''
                activation function goes here and set the 
                result to layer.nodes

                tergantung activation layer nya apa,
                bisa jadi perlu looping karena tiap node butuh perhitungan masing-masing
                atau bisa serentak
                '''
                layer.activate()
                    
                # layer.nodes menjadi list[float] setelah fungsi aktivasi
                ''''hasil aktivasi dijadikan sebagai input pada layer di atasnya
                perlu ditambahkan '1' sebagai bias untuk hidden layer'''
                if(j == network_depth-2):
                    masukan_temp = layer.nodes
                    break
                else:
                    temp = [0] * (layer._width+1)
                    temp[0] = 1
                    for w in range(layer._width):
                        temp[w+1] = layer.nodes[w]
                    masukan_temp = temp
                
            hasil[i] = masukan_temp
        